# Lab 3 - Using Tools

## Introduction

In this lab, you will learn how to add tools to your agents to help them achieve their goals. This includes how they can be utilized to improve agent performance.


## What are we going to cover in this notebook?

In this notebook, we will cover the following topics:

1. Available tools
2. Instanciating tools
3. Adding tools to agents
4. Example Use Case

## Libraries

First, let's import the required libraries, as we did in the previous lab.

In [1]:
from crewai import Agent, Task, Crew, Process
from IPython.display import Markdown

## What are Tools?

A [tool](https://docs.crewai.com/core-concepts/Tools/) in CrewAI is a skill or function that agents can utilize to perform various actions. This enabling everything from simple searches to complex interactions.

### Benefits of Using Tools

- **Utility**: Enables tasks like web searching, data analysis, content generation, and agent collaboration.
- **Integration**: Seamlessly integrates tools into agents' workflows, boosting capabilities.
- **Customizability**: Allows for custom tool development or leveraging existing ones to cater to specific needs.
- **Error Handling**: Implements robust error handling mechanisms for smooth operation.
- **Caching Mechanism**: Optimizes performance and reduces redundant operations through intelligent caching.

### Tool Providers

You can utilize tools provided by [CrewAI](https://docs.crewai.com/core-concepts/Tools/#available-crewai-tools), as well as [LangChain](https://python.langchain.com/v0.2/docs/integrations/tools/). Custom tools can also be created (we'll check this in the next lab)!

## Example: Programming support

In this example we'll create a team of agents that will try to solve programming doubts. To do so, we will first instantiate a tool that performs scraping of a website. This tool, `ScrapeWebsiteTool`, will allow the agents to perform searches on the website to try and solve the doubts the user might have.

Some tools will require to set some parameters to initialise them. In our case, we'll need to configure them to use Ollama and our local models. In the case of the ScrapeWebsiteTool as well as most tools, we have to define a LLM model and an embedding model.

- **LLM**: `llama3.1:latest`
- **Embedder**: `mxbai-embed-large:latest`

For this specific tool we can also provide a URL to the docs (`website_url`) so that the tool focuses on searching on that specific documentation. We will not provide it beforehand, because we want to make our agents capable of solving multiple problems. Instead, we will provide some references at runtime.

#### Different Ways to Give Agents Tools

- **Agent Level**: The Agent can use the Tool(s) on any Task it performs.
- **Task Level**: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

In [2]:
from crewai_tools import ScrapeWebsiteTool

docs_search = ScrapeWebsiteTool(
    config=dict(
        llm=dict(
            provider='ollama',
            config=dict(
                model='llama3.1:latest',
            ),
        ),
        embedder=dict(
            provider='ollama',
            config=dict(
                model='mxbai-embed-large:latest',
            ),
        ),
    )
)

## Creating Agents

A proposal for the agents that we could use to solve this problem is the following one:

### **Programmer Agent**

| **Attribute** | **Value** |
| --- | --- |
| **Name** | Programmer Agent |
| **Role** | Code Generation and Guidance Expert |
| **Goal** | Provide high-quality code snippets and guidance on programming concepts specific to {library}, helping a user solving the following doubt regarding the code: {doubt}. With expertise in {library}, this agent is well-equipped to handle complex coding tasks. |
| **Backstory** | This agent has extensive experience with various programming languages and frameworks, having worked on numerous projects throughout its development, particularly focusing on {library}. Its knowledge is up-to-date, reflecting the latest advancements in software engineering within the context of {library}. Also expert in solving user doubts in the context of {library}. |

### **Quality Assurance Agent**

| **Attribute** | **Value** |
| --- | --- |
| **Name** | Quality Assurance Agent |
| **Role** | Code Review and Quality Assurance Specialist |
| **Goal** | Review the response of the Programmer to a given doubt. Ensure code accuracy and correctness within the {library} framework, identifying potential bugs or areas for improvement to help users produce high-quality software. This agent excels at reviewing code for {library}, providing actionable feedback on best practices and efficiency improvements. |
| **Backstory** | This agent has a proven track record in software quality assurance, having reviewed countless lines of code and provided expert feedback on the implementation details specific to {library}. Its expertise is sought after by many projects, demonstrating its value in ensuring robust and reliable software products within the context of {library}. Make sure that the answer provided by the Programmer is correct and accurate. |


### **Documentation Expert Agent**

| **Attribute** | **Value** |
| --- | --- |
| **Name** | Documentation Expert Agent |
| **Role** | Documentation Standards and Best Practices Expert |
| **Goal** | Add explanations and references to the solution provided by the Quality Assurance. Provide guidance on documentation standards, conventions, and best practices for specific programming languages or frameworks, ensuring the answer given to users is clear and concise and provides explanation of the reasoning behind that solution, also referencing the code documentation within the {library} context. This agent is particularly skilled in providing informative answers to specific doubts users might have on {library}. |
| **Backstory** | This agent has in-depth knowledge of various documentation formats and styles, having worked closely with developers to create detailed documentation for numerous projects within the scope of {library}, highlighting best practices for API design, coding standards, and user interface guidelines. Its expertise is valuable in helping users with their doubts on software within the context of {library}. |

### Exercise

1. Check the provided agent examples
2. Create the three agents
3. Feel free to modifiy the agents description

Examples are provided below, do not check them before trying to implement it by yourself!

In [3]:
# Programmer agent



In [4]:
# QA agent



In [5]:
# Documentation agent



In [6]:
programmer_agent = Agent(
    role='Code Generation and Guidance Expert',
	goal=(
        "Provide high-quality code snippets and guidance on programming concepts specific to "
        "{library}, helping a user solving the following doubt regarding the code: {doubt}. "
        "With expertise in {library}, this agent is well-equipped to handle complex coding tasks."
    ),
	backstory=(
		"This agent has extensive experience with various programming languages and frameworks, having "
        "worked on numerous projects throughout its development, particularly focusing on {library}. Its "
        "knowledge is up-to-date, reflecting the latest advancements in software engineering within the "
        "context of {library}. Also expert in solving user doubts in the context of {library}."
	),
	allow_delegation=False,
	verbose=True,
    llm='ollama/llama3.1'
)

In [7]:
qa_agent = Agent(
    role='Code Review and Quality Assurance Specialist',
	goal=(
        "Review the response of the Programmer to a given doubt. Ensure code accuracy and correctness "
        "within the {library} framework, identifying potential bugs or areas for improvement to help "
        "users produce high-quality software. This agent excels at reviewing code for {library}, "
        "providing actionable feedback on best practices and efficiency improvements."
    ),
	backstory=(
		"This agent has a proven track record in software quality assurance, having reviewed countless "
        "lines of code and provided expert feedback on the implementation details specific to {library}. "
        "Its expertise is sought after by many projects, demonstrating its value in ensuring robust and "
        "reliable software products within the context of {library}. Make sure that the answer provided by "
        "the Programmer is correct and accurate."
	),
	allow_delegation=False,
	verbose=True,
    llm='ollama/llama3.1'
)

In [8]:
documentation_agent = Agent(
    role='Documentation Standards and Best Practices Expert',
	goal=(
        "Add explanations and references to the solution provided by the Quality Assurance. Provide guidance "
        "on documentation standards, conventions, and best practices for specific programming languages or "
        "frameworks, ensuring the answer given to users is clear and concise and provides explanation of the "
        "reasoning behind that solution, also referencing the code documentation within the {library} context. "
        "This agent is particularly skilled in providing informative answers to specific doubts users might have "
        "on {library}."
    ),
	backstory=(
		"This agent has in-depth knowledge of various documentation formats and styles, having worked closely with "
        "developers to create detailed documentation for numerous projects within the scope of {library}, highlighting "
        "best practices for API design, coding standards, and user interface guidelines. Its expertise is valuable in "
        "helping users with their doubts on software within the context of {library}."
	),
	allow_delegation=False,
	verbose=True,
    llm='ollama/llama3.1'
)

## Creating Tasks

Next, we'll create the tasks. One per agent, similarly to how we did in the previous lab.

We are providing some {sources} which the agent (hopefully) will use to search for a solution.

| **Agent** | **Task** | **Description** | **Expected Output** |
| --- | --- | --- | --- |
| **Programmer Agent** | Solve Problem | 1. Thoroughly understand the given problem statement, including any constraints or requirements. <br> 2. Gather some information from sources such as {sources}. <br> 3. Based on expertise, devise a well-thought-out solution that addresses the problem, considering factors like complexity, readability, and efficiency. <br> 4. Implement the solution in the required library {library}, ensuring that it is properly formatted, tested, and meets all specified requirements. | A well-documented and formatted code snippet that accurately solves the problem. A brief explanation of the solution approach used. |

| **Agent** | **Task** | **Description** | **Expected Output** |
| --- | --- | --- | --- |
| **Quality Assurance Specialist Agent** | Review Solution | 1. Receive the solution from the Programmer for review. <br> 2. Thoroughly inspect the code snippet, checking for any potential bugs or areas for improvement. <br> 3. Based on the inspection, identify any issues found in the code and suggest improvements. <br> 4. Improve the solution by solving issues if they were found. | A well-documented and formatted code snippet that accurately solves the problem. A brief explanation of the solution approach used. |

| **Agent** | **Task** | **Description** | **Expected Output** |
| --- | --- | --- | --- |
| **Documentation Expert Agent** | Create Documentation | 1. Receive the corrected solution from the Quality Assurance Specialist. <br> 2. Based on this solution, develop a comprehensive guide explaining how it has been solved. <br> 3. Ensure that the guide is well-organized, making it easy for user to understand the proposed solution. | A clear and concise guide that explains the solution and provides step-by-step instructions. |

**Note**: we'll add the web scraper tool to the problem solving task. The programmer agent will be able to check the docs.

### Exercise

1. Check the provided task examples
2. Create the three tasks
3. Feel free to modifiy the tasks description

Examples are provided below, do not check them before trying to implement it by yourself!

In [9]:
# Solve problem task



In [10]:
# Review solution task



In [11]:
# Create documentation task



In [12]:
solve_problem = Task(
    description=(
        "1. Thoroughly understand the given problem statement, including any constraints or requirements.\n"
        "2. Gather some information from sources such as {sources}.\n"
        "3. Based on expertise, devise a well-thought-out solution that addresses the problem, considering "
        "factors like complexity, readability, and efficiency.\n"
        "4. Implement the solution in the required library {library}, ensuring that it is properly formatted, "
        "tested, and meets all specified requirements."
    ),
    expected_output=(
	    "A well-documented and formatted code snippet that accurately solves the problem. "
        "A brief explanation of the solution approach used."
    ),
	tools=[docs_search],
    agent=programmer_agent,
)

In [13]:
review_solution = Task(
    description=(
        "1. Receive the solution from the Programmer for review.\n"
        "2. Thoroughly inspect the code snippet, checking for any potential bugs or areas for improvement.\n"
        "3. Based on the inspection, identify any issues found in the code and suggest improvements.\n"
        "4. Improve the solution by solving issues if they were found."
    ),
    expected_output=(
	    "A well-documented and formatted code snippet that accurately solves the problem. "
        "A brief explanation of the solution approach used."
    ),
    agent=qa_agent,
)

In [14]:
create_documentation = Task(
    description=(
        "1. Receive the corrected solution from the Quality Assurance Specialist.\n"
        "2. Based on this solution, develop a comprehensive guide explaining how it has been solved.\n"
        "3. Ensure that the guide is well-organized, making it easy for user to understand the proposed solution."
    ),
    expected_output=(
	    "A clear and concise guide that explains the solution and provides step-by-step instructions."
    ),
    agent=documentation_agent,
)

## Creating the Crew

Now we will create the crew with sequential processes. We will set an additional parameter, `full_output=True`, to obtain the whole answer provided by the agents.

### Exercise

Create the Crew.

An example is provided below, do not check it before trying to implement it by yourself!

In [15]:
# Crew creation



In [16]:
crew = Crew(
  agents=[programmer_agent, qa_agent, documentation_agent],
  tasks=[solve_problem, review_solution, create_documentation],
  full_output=True
)

## Running the Crew - The Custom Tool problem

We will ask the MAS how could we create a custom tool in CrewAI (Next lab's topic!).

### Exercise

Check the execution logs and final output. Does it make sense? Could it be helpful to implement a new custom tool?

In [17]:
inputs = {
    'library': 'CrewAI',
    'doubt': 'I need to create and use a Custom Tool in CrewAI by subclassing BaseTool. Can you provide a guide on how to do it?',
    'sources': 'https://docs.crewai.com/how-to/Create-Custom-Tools/'
}
result = crew.kickoff(inputs=inputs)

# Agent: Code Generation and Guidance Expert
## Task: 1. Thoroughly understand the given problem statement, including any constraints or requirements.
2. Gather some information from sources such as https://docs.crewai.com/how-to/Create-Custom-Tools/.
3. Based on expertise, devise a well-thought-out solution that addresses the problem, considering factors like complexity, readability, and efficiency.
4. Implement the solution in the required library CrewAI, ensuring that it is properly formatted, tested, and meets all specified requirements.


# Agent: Code Generation and Guidance Expert
## Thought: Thought: I need to create and use a Custom Tool in CrewAI by subclassing BaseTool. To do this, I should gather information from relevant sources.
## Using tool: Read website content
## Tool Input: 
{
  "website_url": "https://docs.crewai.com/how-to/Create-Custom-Tools/"
}
## Tool Output: 
Creating and Utilizing Tools in crewAI - crewAI
 Skip to content
 crewAI
 Creating and Utilizing Tools 

Display the final result as Markdown.

In [18]:
Markdown(result.raw)

**Comprehensive Guide: Resolving Issues in Tool Functions**

### Introduction

The provided code snippet showcases the creation of custom tool functions within the CrewAI framework. However, upon review, potential bugs and areas for improvement were identified. This guide outlines the steps taken to resolve these issues, ensuring that tool functions operate correctly within the CrewAI context.

### Step 1: Importing Necessary Modules from the CrewAI Library

The code begins by importing necessary modules from the crewai library using the following lines:
```python
from crewai_tools import BaseTool
from crewai_tools import tool
```
These imports enable the creation of custom tools and the use of the `tool` decorator.

### Step 2: Subclassing `BaseTool` to Create a Custom Tool

The code defines a custom tool named `MyCustomTool` by subclassing `BaseTool`:
```python
class MyCustomTool(BaseTool):
    name: str = "Name of my tool"
    description: str = "What this tool does. It's vital for effective utilization."
```
This subclassing allows for the creation of a custom tool with its own properties and methods.

### Step 3: Defining the `_run` Method within the Custom Tool Class

The `_run` method is defined within the `MyCustomTool` class to handle the actual logic of the tool:
```python
def _run(self, argument: str) -> str:
    """This is where your custom logic goes."""
    if not isinstance(argument, str):
        raise TypeError("Argument must be a string")
    return "Tool's result"
```
In this step, type checking was added to ensure that the `argument` passed to the `_run` method is indeed a string. If the argument is not a string, a `TypeError` is raised.

### Step 4: Using the `tool` Decorator to Define Simple Tool Functions

The code demonstrates the use of the `tool` decorator to define simple tool functions:
```python
@tool("My Simple Tool")
def my_simple_tool(question: str) -> str:
    """A brief description of this tool."""
    if not isinstance(question, str):
        raise TypeError("Question must be a string")
    return "Tool output from decorated function."
```
Similar to the `_run` method, type checking is added to ensure that the `question` passed to `my_simple_tool` is indeed a string.

### Step 5: Defining a Custom Caching Strategy

The code defines a custom caching strategy within the `my_cache_strategy` function:
```python
def my_cache_strategy(arguments: dict, result: str) -> bool:
    # Custom caching logic here
    if not isinstance(arguments, dict):
        raise TypeError("Arguments must be a dictionary")
    return True  # Modified to always return True for simplicity
```
Type checking is added to ensure that the `arguments` passed to `my_cache_strategy` is indeed a dictionary. However, it's essential to note that the return value of this function should be modified according to your custom caching logic.

### Step 6: Applying the Custom Caching Strategy

The code applies the custom caching strategy to an existing tool function named `cached_tool`:
```python
@tool("Tool with Caching")
def cached_tool(argument: str) -> str:
    """A brief description of this tool."""
    if not isinstance(argument, str):
        raise TypeError("Argument must be a string")
    result = "Cacheable result"
    cached_tool.cache_function = my_cache_strategy
    return result
```
Again, type checking is added to ensure that the `argument` passed to `cached_tool` is indeed a string.

### Step 7: Example Usage of Custom Tool Functions

The code demonstrates example usage of the custom tool functions within a CrewAI context:
```python
crewai_obj = crewai()  # Assuming crewai is defined elsewhere
tool_instance_1 = crewai_obj.create_tool(MyCustomTool)
try:
    result_from_my_custom_tool = tool_instance_1.run(argument="Hello world!")
except TypeError as e:
    print(f"Error: {e}")

tool_instance_2 = crewai_obj.create_tool(my_simple_tool)
result_from_decorated_function = tool_instance_2.run(question="How are you?")

tool_instance_3 = crewai_obj.create_tool(cached_tool)
try:
    result_with_caching = tool_instance_3.run(argument="Test caching")
except TypeError as e:
    print(f"Error: {e}")
```
This example showcases the proper usage of custom tool functions within a CrewAI context, including error handling for potential type-related issues.

By following these steps and implementing type checking where necessary, the corrected code snippet ensures that tool functions operate correctly within the CrewAI framework.

## Useful resources

- [CrewAI Docs](https://docs.crewai.com/)
- [CrewAI Repository](https://github.com/crewAIInc/crewAI)
- [Langchain Docs](https://python.langchain.com/v0.2/docs/introduction/)
- [CrewAI Tools](https://docs.crewai.com/core-concepts/Tools/)
- [CrewAI Custom Tools](https://docs.crewai.com/how-to/Create-Custom-Tools/)
- [CrewAI Tools Repository](https://github.com/crewAIInc/crewAI-tools)
- [LangChain Tools](https://python.langchain.com/v0.2/docs/integrations/tools/)